In [85]:
from cs336_basics.transformer import Transformer
from cs336_basics.train import AdamW, get_batch, gradient_clipping, get_lr_schedule,CrossEntropyLossWithLogits, load_checkpoint, save_checkpoint
from config import Config
import tiktoken
from torch import nn
import torch
from pathlib import Path

cfg = Config()
model = Transformer(
        cfg.d_model, cfg.num_heads, cfg.d_ff, cfg.context_length, cfg.rope_theta, cfg.vocab_size, cfg.num_layers
    ).to(cfg.device)
load_checkpoint(Path(cfg.ckpt_path) / "dev_0099.pt", model, optimizer=None)
tokenizer = tiktoken.get_encoding("gpt2")


In [172]:
@torch.no_grad()
def decode(model: nn.Module, input_ids: torch.Tensor, max_new_tokens: int, temperature: float, p: float) -> torch.Tensor:
    model.eval()
    token_generated = 0
    while token_generated < max_new_tokens:
        logits = model(input_ids)[:, -1, :]
        logits = logits / temperature  
        probs = torch.softmax(logits, dim = 1)          
        if 0.0 < p < 1.0:
            sorted, indices = torch.sort(probs, descending=True)
            cumsum = torch.cumsum(sorted, dim=1)
            trimmed = cumsum > p
            trimmed[: , 0] = False
            sorted[trimmed] = 0
            next_token_id = torch.multinomial(sorted, num_samples=1)
            next_token = indices.gather(1, next_token_id)
        else:
            next_token = torch.multinomial(probs, num_samples=1)
        if next_token.item() == 50256:
            break
        input_ids = torch.cat((input_ids, next_token), dim=1)
        token_generated += 1
    return input_ids
prompt = "Once upon a time there was a little boy named Ben. Ben loved to explore"
input_ids = tokenizer.encode(prompt)
input_ids = torch.as_tensor(input_ids, dtype=torch.int64).unsqueeze(0)
output = decode(model, input_ids, max_new_tokens=100, temperature=0.5, p = 0.9).squeeze().tolist()
print(tokenizer.decode(output))

Once upon a time there was a little boy named Ben. Ben loved to explore the woods. The cat was very happy.

